In [1]:
import pandas as pd
import re
import numpy as np
import os
from datetime import timedelta
from time_dissection_func import get_events_prior_to

pd.set_option("display.max_rows", None)

In [2]:
output_dir = "../../../results/datasets/time_dissection"

# Read raw data

In [3]:
full_data = pd.read_csv("../../../results/datasets/allFeatures_fullDataset.csv", low_memory=False)
print(full_data.shape)
variables_per_dataset = pd.read_csv("../../../data/VARIABLES_timepoints_GASpred.csv", header=0, index_col=0)
print(variables_per_dataset.shape)

(409, 2488)
(2410, 12)


# Prepare BL datast (CODE: 4)

In [4]:
bl_indx = (variables_per_dataset["ICU-BL"][variables_per_dataset["ICU-BL"] == 4]).index
print(bl_indx)
df_bl = full_data.loc[:,bl_indx]

###### SAVE dataset yes|no ######
save = True
#################################

# 
time0 = "date_of_data_2"

# create backup pf original dataset
df_bl_original = df_bl.copy()

Index(['PATIENT_ID', 'CENTER_CODE', 'SEX', 'AGE', 'WEIGHT', 'HEIGHT',
       'DATE_FIRST_ADMISSION', 'HOSPITAL_FORST_ADMIS', 'DATE_DIAGNOSIS',
       'DATE_SPEC_HOSP',
       ...
       'SBE_BL', 'lactate_BL', 'glucose_BL', 'crystalloids_BL', 'colloids_BL',
       'albumin_BL', 'fluid_other_BL', 'blood_products_BL', 'nutrition_BL',
       'accum_fluids_BL'],
      dtype='object', length=877)


## Bacterial specimen isolated - tissue

Apply to columns:

       - sample_{1-25}
       - sample_site_
       - date_specimen_sample_
       - collected_during_surgery_{1-25}
       - collection_mode_
  

In [5]:
### BACTERIAL SPECIES TISSUE ###

time_prior_event = "date_specimen_"
bact_columns = ["sample_", "sample_site_", "date_specimen_sample_", 
                "collected_during_surgery_", "collection_mode_"]


df_bl = get_events_prior_to(df_bl, time0, time_prior_event, bact_columns, hard_impute=True)   

# Check results
orig_sample = df_bl_original.loc[:10,"sample_2"]
orig_blood= df_bl_original.loc[:10,"blood_2"]
orig_skin = df_bl_original.loc[:10,"skin_bulla_2"]

tissue_sample = df_bl.loc[:10,"sample_2"]
tissue_blood= df_bl.loc[:10,"blood_2"]

## Bacterial specimen isolated - blood

Apply to columns:

       - blood_{1-25}
       - sample_site_blood_
       - date_blood_sample_
       - collect_surg_blood_
       - collection_mode_blood_
       

In [6]:
## BACTERIAL SPECIES BLOOD ###
#time0 = "???" defined above
time_prior_event = "date_blood_sample_"
blood_columns = ["blood_", "sample_site_blood_", "date_blood_sample_",
                 "collect_surg_blood_","collection_mode_blood_","collected_during_surgery_"]
 
df_bl = get_events_prior_to(df_bl, time0, time_prior_event, blood_columns, hard_impute=True)  

# Check results
blood_sample = df_bl.loc[:10,"sample_2"]
blood_blood= df_bl.loc[:10,"blood_2"]

## Surgery 

Apply to columns:

       - surgery_
       - hospital_surgery_
       - time_closure_
       - time_incision_
       - patient_files_surgery_
       - contact_surgeon_surgery_
       - skin_bulla_
       - skin_erythem_
       - subcutis_dissolved_
       - subcutis_deliquescent_
       - fascia_greyish_
       - fascia_fingertest_
       - fascia_dishwater_
       - muscle_contractions_
       - muscle_seeping_
       - muscle_brownish_
       - frank_pus_
       - debridement_
       - fasciotomy_
       - amputate_upper_arm_
       - amputate_lower_arm_      
       - amputate_hand_   
       - amputate_fingers_   
       - amputate_upper_leg_
       - amputate_lower_leg_ 
       - amputate_foot_   
       - amputate_toe_
       - amputate_penis_     
       - surgery_blood_
       - est_blood_loss_
       - description_blood_loss_
       - vac_
       - skin_defect_pct_
       - biopsy_tissue_

In [7]:
## BACTERIAL SPECIES BLOOD ###
#time0 = "???" defined above
time_prior_event = "time_incision_"
surgery_columns = ["surgery_",
                    "hospital_surgery_",
                    "time_closure_",
                    "time_incision_",
                    "patient_files_surgery_",
                    "contact_surgeon_surgery_",
                    "skin_bulla_",
                    "skin_erythem_",
                    "subcutis_dissolved_",
                    "subcutis_deliquescent_",
                    "fascia_greyish_",
                    "fascia_fingertest_",
                    "fascia_dishwater_",
                    "muscle_contractions_",
                    "muscle_seeping_",
                    "muscle_brownish_",
                    "frank_pus_",
                    "debridement_",
                    "fasciotomy_",
                    "amputate_upper_arm_",
                    "amputate_lower_arm_",
                    "amputate_hand_"   ,
                    "amputate_fingers_"   ,
                    "amputate_upper_leg_",
                    "amputate_lower_leg_" ,
                    "amputate_foot_"   ,
                    "amputate_toe_",
                    "amputate_penis_"     ,
                    "surgery_blood_",
                    "est_blood_loss_",
                    "description_blood_loss_",
                    "vac_",
                    "skin_defect_pct_",
                    "biopsy_tissue_"]

df_bl = get_events_prior_to(df_bl, time0, time_prior_event, surgery_columns, hard_impute=True)  

# Check results
surg_sample = df_bl.loc[:10,"sample_2"]
surg_blood= df_bl.loc[:10,"blood_2"]
surg_skin = df_bl.loc[:10,"skin_bulla_2"]

## Save Dataset

In [8]:
if save == True:
    df_bl.to_csv(os.path.join(output_dir, "BL_dataset.csv"), index=False)

## Check results

In [ ]:
# ### Check results ###

# print("original data: \n")
# print(orig_sample)
# print(orig_blood)
# print(orig_skin)

# print("\n\nafter tissue parsing: \n")
# print(tissue_sample)
# print(tissue_blood)

# print("\n\nafter blood parsing: \n")
# print(blood_sample)
# print(blood_blood)

# print("\n\nafter surg parsing: \n")
# print(surg_sample)
# print(surg_blood)
# print(surg_skin)

In [ ]:
# a = df_bl_original.isna().sum().to_list()
# b = df_bl.isna().sum().to_list()

# for i in range(len(a)):
#     print(df_bl_original.columns[i],"\t\t\t",a[i],"\t",b[i])

In [9]:
#df_bl.isna().sum()
print(df_bl.shape)
df_bl_original.shape

(409, 1332)


(409, 1332)

In [10]:
df_bl.isna().sum()

PATIENT_ID                       0
CENTER_CODE                      0
SEX                              0
AGE                              0
WEIGHT                           2
HEIGHT                           7
DATE_FIRST_ADMISSION             0
HOSPITAL_FORST_ADMIS             1
DATE_DIAGNOSIS                   0
DATE_SPEC_HOSP                   0
DATE_ICU_ADMISSION               0
HEAD_NECK_ARRIVAL                0
UPPER_ARM_ARRIVAL                0
LOWER_ARM_ARRIVAL                0
HAND_ARRIVAL                     0
FINGER_ARRIVAL                   0
THORAX_ARRIVAL                   0
ABDOMEN_ARRIVAL                  0
ANOGENITAL_ARRIVAL               0
UPPER_LEG_ARRIVAL                0
LOWER_LEG_ARRIVAL                0
FOOT_ARRIVAL                     0
TOE_ARRIVAL                      0
ALCOHOL                         97
SMOKING                         63
PREV_NSTI                        0
SURGERY_BEFORE                   0
AIDS                             0
IMMUNODEF           